------------------------------------------------

Necessary imports

In [1]:
import pandas as pd
import os 
import glob
! pip install pandasql
from pandasql import sqldf
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
current_directory = os.getcwd()
mydir = current_directory
file_list = glob.glob(mydir + "/*.csv")

In [3]:
file_list

['C:\\Users\\junin\\Documents\\Data-Science\\MIMICAnalysis\\admissions.csv',
 'C:\\Users\\junin\\Documents\\Data-Science\\MIMICAnalysis\\d_labitems.csv',
 'C:\\Users\\junin\\Documents\\Data-Science\\MIMICAnalysis\\edstays.csv',
 'C:\\Users\\junin\\Documents\\Data-Science\\MIMICAnalysis\\labevents.csv']

In [4]:
list_df = []
for i in file_list:
    df = pd.read_csv(i)
    list_df.append(df)

In [5]:
df_admissions = list_df[0]
df_labitems = list_df[1]
df_edstays = list_df[2]
df_labevents = list_df[3]

## Reading the data

In [6]:
df_admissions.head(5)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,10004235,24181354,2196-02-24T14:38:00,2196-03-04T14:02:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2196-02-24T12:15:00,2196-02-24T17:07:00,0
1,10009628,25926192,2153-09-17T17:08:00,2153-09-25T13:20:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO,NaN,NaN,0
2,10018081,23983182,2134-08-18T02:02:00,2134-08-23T19:35:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17T16:24:00,2134-08-18T03:15:00,0
3,10006053,22942076,2111-11-13T23:39:00,2111-11-15T17:20:00,2111-11-15T17:20:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaN,NaN,1
4,10031404,21606243,2113-08-04T18:46:00,2113-08-06T20:57:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaN,NaN,0


In [7]:
df_edstays.head(5)

,subject_id,hadm_id,stay_id,intime,outtime
0,10014729,23300884.0,32435236,2125-03-19T12:36:00,2125-03-19T16:59:47
1,10004235,24181354.0,36583389,2196-02-24T12:15:00,2196-02-24T17:07:00
2,10026255,22059910.0,30926886,2201-07-07T12:31:00,2201-07-07T19:40:00
3,10026255,20437651.0,36948716,2200-09-17T18:38:00,2200-09-18T00:57:00
4,10007058,22954658.0,35770266,2167-11-07T17:57:00,2167-11-07T20:22:00


In [8]:
df_labevents.head(5)

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,183881,10014354,29600294.0,1808066,51277,2148-08-16 00:00:00,2148-08-16 01:30:00,15.4,15.4,%,10.5,15.5,NaN,ROUTINE,NaN
1,183883,10014354,29600294.0,1808066,51301,2148-08-16 00:00:00,2148-08-16 01:30:00,20.3,20.3,K/uL,4.0,10.0,abnormal,ROUTINE,NaN
2,183888,10014354,29600294.0,1808066,52167,2148-08-16 00:00:00,2148-08-16 01:30:00,49.7,49.7,fL,35.1,46.3,abnormal,ROUTINE,NaN
3,183870,10014354,29600294.0,1808066,51249,2148-08-16 00:00:00,2148-08-16 01:30:00,31.1,31.1,g/dL,32.0,37.0,abnormal,ROUTINE,NaN
4,183865,10014354,29600294.0,1808066,51222,2148-08-16 00:00:00,2148-08-16 01:30:00,9.2,9.2,g/dL,13.7,17.5,abnormal,ROUTINE,NaN


In [9]:
df_labitems.head(5)

,itemid,label,fluid,category,loinc_code
0,52016,Abz,Blood,Blood Gas,NaN
1,52017,"Albumin, Blood",Blood,Blood Gas,NaN
2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,NaN
3,52018,Assist/Control,Blood,Blood Gas,NaN
4,50802,Base Excess,Blood,Blood Gas,NaN


 #  <span style="color:blue">  1.	What is the percentage of patients admitted to the hospital that have never been to the ED? </span>

In [10]:
# We have 100 patients in admissions table 
total_patients = len(df_admissions['subject_id'].unique())
print(total_patients)

100


In [11]:
#First i got all ED admissions
df_emergency_room = df_admissions.query('admission_location=="EMERGENCY ROOM"')

In [12]:
#After that I removed all admissions that contain the type ED from the admissions table
df_remove = df_admissions[~df_admissions.subject_id.isin(df_emergency_room.subject_id)]

In [13]:
#In the sequence of the table that contains the different admissions of the ED type, the filter of single patients was performed
total_dif = len(df_remove['subject_id'].unique())

In [14]:
#Calculation: of patients admitted to the hospital that have never been to the ED
porcent = (total_dif/total_patients)*100
print(f'Percentage of patients admitted to the hospital that have never been to the ED is {porcent} %')

Percentage of patients admitted to the hospital that have never been to the ED is 39.0 %


 #  <span style="color:blue">   2.		For each patient, generate a json with the hospital admission data. </span>

* My primary interpretation was to store all patients in a single JSON file, in which they can be identified by their respective ids/subject_id.


* However, there is also the possibility of creating a JSON file for each patient. This can be easily performed with simple coding changes.

##### I created two JSON files. In simple_admission.json only the data contained in the admissions table is present.

In [15]:
list_att = df_admissions.columns[1:]
d = (df_admissions.groupby(['subject_id'])[list_att]
       .apply(lambda x: x.to_dict('r')).to_json())
simple_json_object = json.loads(d)

In [17]:
for key in simple_json_object.keys():
    json_patient = simple_json_object[key]
    with open(f'{mydir}\\simple_admission_patients\\{key}.json', 'w') as fp:
        json.dump(json_patient, fp)

In [18]:
with open('simple_admission.json', 'w') as fp:
    json.dump(simple_json_object, fp)

In [19]:
##### The full admission.json contains all the exam results, stay that inherit the admissions of the patients through the relationships of the tables: df_edstays, df_labevents, and df_labitems.

In [20]:
query = """SELECT   df_admissions.subject_id,	df_admissions.hadm_id,	df_admissions.admittime,	df_admissions.dischtime,	df_admissions.deathtime,	df_admissions.admission_type,	df_admissions.admission_location,	df_admissions.discharge_location,	df_admissions.insurance,	df_admissions.language,	df_admissions.marital_status,	df_admissions.ethnicity,	df_admissions.edregtime,	df_admissions.edouttime,	df_admissions.hospital_expire_flag,

df_edstays.stay_id, df_edstays.intime stay_intime, df_edstays.outtime stay_outime,

df_labevents.labevent_id, df_labevents.specimen_id labE_specimen_id, df_labevents.itemid labE_itemid,	df_labevents.charttime labE_charttime,	df_labevents.storetime labE_storetime,	df_labevents.value labE_value,	df_labevents.valuenum labE_valuenum,	df_labevents.valueuom labE_valueuom,	df_labevents.ref_range_lower labE_ref_range_lower,	df_labevents.ref_range_upper labE_ref_range_upper,	df_labevents.flag labE_flag,	df_labevents.priority labE_priority,	df_labevents.comments labE_comments,


df_labitems.label labI_label,	df_labitems.fluid labI_fluid,	df_labitems.category labI_category,	df_labitems.loinc_code labI_loinc_code


from  df_admissions 


LEFT JOIN df_edstays on df_edstays.hadm_id = df_admissions.hadm_id

LEFT JOIN df_labevents on df_labevents.hadm_id = df_admissions.hadm_id

INNER JOIN df_labitems on df_labitems.itemid = df_labevents.itemid;

"""

df_query_resul =sqldf(query, globals())

In [21]:
list_att = df_query_resul.columns[1:]
d = (df_query_resul.groupby(['subject_id'])[list_att]
       .apply(lambda x: x.to_dict('r')).to_json())
full_json_object = json.loads(d)

In [22]:
for key in full_json_object.keys():
    json_patient = full_json_object[key]
    with open(f'{mydir}\\full_admission_patients\\{key}.json', 'w') as fp:
        json.dump(json_patient, fp)

In [23]:
with open('full_admission.json', 'w') as fp:
    json.dump(full_json_object, fp)